In [143]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed, CreateAccountParams, create_account
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
from solana.system_program import SYS_PROGRAM_ID
from spl.token.client import Token
from solana import keypair
from solana.publickey import PublicKey
from solana.sysvar import SYSVAR_RENT_PUBKEY


TOKEN_PROGRAM_ID: PublicKey = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")

first_key_in_key_folder = glob.glob("keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])
deployed_program_key = glob.glob("target/deploy/*.json")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
program_id = deployed_program_key_account.public_key
http_client = Client("https://api.devnet.solana.com")
payer_public_key = payer_loaded_account.public_key



# todo: change program_id to system program, change data size =0, change lampord for both alice and bob
tx = Transaction()
alice_keypair = solana.keypair.Keypair.generate()
alice_pubkey = alice_keypair.public_key
# data_size = 10
# lamports = http_client.get_minimum_balance_for_rent_exemption(data_size)["result"]
# transaction_instruction = create_account(
#         params=CreateAccountParams(
#             from_pubkey=payer_public_key, new_account_pubkey=alice_pubkey,
#             lamports=lamports, space=data_size, program_id=PublicKey(1)
#         )
#     )

# tx = tx.add(transaction_instruction)
# transaction_results = http_client.send_transaction(tx, *[payer_loaded_account,alice_keypair])



tx = Transaction()
bob_keypair = solana.keypair.Keypair.generate()
bob_pubkey = bob_keypair.public_key
# data_size = 0
# lamports = http_client.get_minimum_balance_for_rent_exemption(data_size)["result"]
# transaction_instruction = create_account(
#         params=CreateAccountParams(
#             from_pubkey=payer_public_key, new_account_pubkey=bob_pubkey,
#             lamports=lamports, space=data_size, program_id=PublicKey(1)
#         )
#     )

# tx = tx.add(transaction_instruction)
# transaction_results = http_client.send_transaction(tx, *[payer_loaded_account,bob_keypair])


print(f"Program ID: {program_id}")
print(f"Payer Account: {payer_public_key}")
print(f"alice Account: {alice_pubkey}")
print(f"bob Account: {bob_pubkey}")

Program ID: GnpKs4JLVt39MfAHjQnXZSxY74nmqeTWTdYiw3FZ5jt1
Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
alice Account: 2Axw99UXXZPkf7h5bos6Gwvtr3BE6dG3C8crxP9Kbzxj
bob Account: 3jLVeXsXDzMkdQ3wcSz8bqFJm7pJPMq86xH8RQchm4gw


In [144]:
# escrow meta-data account
tx = Transaction()
escrow_keypair = solana.keypair.Keypair.generate()
escrow_pubkey = escrow_keypair.public_key
data_size = 100
lamports = http_client.get_minimum_balance_for_rent_exemption(data_size)["result"]
transaction_instruction = create_account(
        params=CreateAccountParams(
            from_pubkey=payer_public_key, new_account_pubkey=escrow_pubkey,
            lamports=lamports, space=data_size, program_id=program_id
        )
    )

tx = tx.add(transaction_instruction)
transaction_results = http_client.send_transaction(tx, *[payer_loaded_account,escrow_keypair])



In [145]:
import time
# http_client.request_airdrop(payer_loaded_account.public_key, 1000000000)
# time.sleep(20)
http_client.request_airdrop(alice_pubkey, 1000000000)
time.sleep(20)
http_client.request_airdrop(bob_pubkey, 1000000000)
print(http_client.get_balance(alice_pubkey))
print(http_client.get_minimum_balance_for_rent_exemption(7)["result"])

{'jsonrpc': '2.0', 'result': {'context': {'slot': 85873550}, 'value': 1000000000}, 'id': 6}
939600


In [146]:
X_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
Y_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )

In [147]:
alice_X_token_account = X_mint_account_address.create_associated_token_account(alice_pubkey)
alice_Y_token_account = Y_mint_account_address.create_associated_token_account(alice_pubkey)
bob_X_token_account = X_mint_account_address.create_associated_token_account(bob_pubkey)
bob_Y_token_account = Y_mint_account_address.create_associated_token_account(bob_pubkey)
# todo, creat seed more specific
vaultx, xseed = PublicKey.find_program_address(seeds=[b"alice"],program_id=program_id)
vaulty, yseed = PublicKey.find_program_address(seeds=[b"bob"],program_id=program_id)
escrow_address, escrow_seed = PublicKey.find_program_address(seeds=[b"escrow"],program_id=program_id)

In [156]:
yseed

255

In [159]:
xseed

251

In [150]:
SYS_PROGRAM_ID

11111111111111111111111111111111

In [155]:
from struct import *
data = pack('BQQ', 1,12,15)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=False),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=payer_public_key, is_signer=True, is_writable=False),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYSVAR_RENT_PUBKEY, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYS_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data, #struct.pack("<LHB",1,1,1) # b58encode(bytes([9] * 7))#b'0001000100010001000100010001',
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[payer_loaded_account])#, *[payer_loaded_account])

{'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: Failed to serialize or deserialize account data: Unknown', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'BorshIoError': 'Unknown'}]}, 'logs': ['Program GnpKs4JLVt39MfAHjQnXZSxY74nmqeTWTdYiw3FZ5jt1 invoke [1]', 'Program log: Step 0', 'Program log: Step 1', 'Program log: Step 2', 'Program log: Step 3', 'Program log: Step 4', 'Program log: Step 7', 'Program log: Step 9', 'Program log: Step 10', 'Program 11111111111111111111111111111111 invoke [2]', 'Program 11111111111111111111111111111111 success', 'Program log: Step 11', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: InitializeAccount', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3412 of 187866 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Step 8', 'Program log: Step 9', 'Program log: Step 10', 'Program 1111111111111111111

Exception: Failed to send transaction

In [142]:
PublicKey(1)

11111111111111111111111111111112